# Backpropagation
----

We have $\begin{eqnarray} 
  a^{l}_j = \sigma\left( \sum_k w^{l}_{jk} a^{l-1}_k + b^l_j \right),
\end{eqnarray}$
- To rewrite this equation in matrix form, we define a weight matrix $w^l$. The entries in the matrix are just the weights connecting to the lth layer of neurons, i.e. entry in jth row and kth column is $w^l_{jk}$.
- The components of the bias vector are just the values $b^l_j$, one component for each neuron in the lth layer. Similarly for $a^l$ vector containing all the activations in a layer.
- Above equation can be written as,
 $\begin{eqnarray} 
  a^{l} = \sigma(w^l a^{l-1}+b^l).
\end{eqnarray}$ in vector form.
- This equation gives a global way of thinking about how the activations in one layer relate to activations in the previous layer. We just apply the weight matrix to the activations, add the bias vector and apply sigmoid function.
- To compute $a^l$, we compute the intermediate quantity $z^l \equiv w^l a^{l-1}+b^l$ along the way. This is called the **weighted input** to the neurons in layer l.

## Assumptions regarding cost function

- The goal of backprop is to compute the partial derivatives with respect to any weight and bias of the cost function.
- First assumption is that, the CF can be written as an average $C = \frac{1}{n} \sum_x C_x$ over cost function $C_x$ for individual examples, x. Therefore, $C_x =
\frac{1}{2} \|y-a^L \|^2$. This holds true for the quadratic CF.

We need this assumption because, backpropagation actually lets us compute partial derivatives for a single training example, then we average over training examples.

- Second assumption is that, the CF can be written as a function of the outputs from the neural network. It is also a function of y, but with a fixed input symbol x, y is also fixed, so it is not something the network learns or we cannot change it by tweaking the weights and biases. Therefore, we regard $C$ as a function of the output activations $a^L$ alone with y merely a parameter.

## The Hadamard product

- Suppose s and t are vectores of same dimension, then we use $s \odot t$ to denote the elementwise product of the two vectors. This is called the Hadamard product.

## Four Fundamental equations behind backpropagation

To compute the partial derivatives, we compute an intermediate quantity $\delta^l_{j}$, which is called the **error** in the jth neuron in the lth layer. Backpropagation will give a procedure to compute the errors and then relate it to the partial derivatives.



1. **Equation for the error in the output layer, $\delta^L$**
$\begin{eqnarray} 
  \delta^L_j = \frac{\partial C}{\partial a^L_j} \sigma'(z^L_j).
\tag{BP1}\end{eqnarray}$
The partial derivative term measures how fast the cost is changing as a function of the jth output activation. If C does not depend on the particular output neuron j, then $\delta$ will be small. The second term measures how fast the activation function, $\sigma$ is changing at $z^L_{j}$.
If we are using the quadratic CF $C = \frac{1}{2} \sum_j (y_j-a^L_j)^2$ and so $\partial C / \partial a^L_j = (a_j^L-y_j)$ is easily computable.

    * BP1 is a component wise expression for $\delta^L$. Matrix form is $\begin{eqnarray}\delta^L = \nabla_a C \odot \sigma'(z^L). \tag{BP1a}\end{eqnarray}$
    
    $\nabla_a C$ is defined to be a vector whose components are the partial derivatives $\partial C / \partial a^L_j$. It is expressing the rate of change of C w.r.t output activations.
     In the case of the quadratic cost we have $\nabla_a C = (a^L-y)$ so the full matrix based form of BP1 is $\begin{eqnarray} 
  \delta^L = (a^L-y) \odot \sigma'(z^L).
\tag{p}\end{eqnarray}$
    

2. **Equation of error $\delta^l$ in terms of the error of next layer, $\delta^{l+1}$**

$\begin{eqnarray} 
  \delta^l = ((w^{l+1})^T \delta^{l+1}) \odot \sigma'(z^l),
\tag{BP2}\end{eqnarray}$

Suppose we know the error $\delta^{l+1}$ at layer l+1. When we apply the transpose of weight matrix, we can think of this as moving the error backward through the network, giving a measure of the error at output of layer l. We then take the Hadamard product. This moves the error backward through the activation function in layer l and giving error $\delta^l$ in the weighted input to layer l. By combining BP1 and BP2 we can compute error for any layer in the network.

3. **Equation for the rate of change of the cost with respect to any bias in the network**

$\begin{eqnarray}  \frac{\partial C}{\partial b^l_j} =
  \delta^l_j.
\tag{BP3}\end{eqnarray}$

The error is equal to the rate of change of $C$ w.r.t the bias. Since using BP1 and BP2, we can compute $\delta^l_j$ , we can rewrite BP3 as $\begin{eqnarray}
  \frac{\partial C}{\partial b} = \delta,
\tag{Q}\end{eqnarray}$

4. **Equation for the rate of change of the cost with respect to any weight in the network**

$\begin{eqnarray}\frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k \delta^l_j. \tag{BP4}\end{eqnarray}$

This tells us how to compute the partial derivatives in terms of $\delta^l$ and $a^{l-1}$. This equation can be written without the index as $\begin{eqnarray}  \frac{\partial C}{\partial w} = a_{\rm in} \delta_{\rm out},\end{eqnarray}$ , where $a_{in}$ is the activation of the neuron input to the weight $w$, and $\delta_{out}$ is the error of the neuron output from the weight $w$.
If $a_{in} \approx 0$ , then the gradient term will also tend to be small. Therefore, the weight learns slowly. Consequence of BP4 is that weights output from low-activation neurons learn slowly.

* BP1 - $\sigma$ function becomes very flat when $\sigma(z^L_j)$ is approximately equal to 0 or 1. When this occurs we will have $\sigma'(z^L_j) \approx 0$. So the weight in the final layer will learn slowly if the output neuron is either low activation or high activation. In this case we can say that the output neuron has saturated and the weight has stopped learning. Same for the biases of output neuron.

* BP2 - The term $\sigma'(z^l)$ in BP2. This means that $\delta^l_j$ is likely to get small if the neuron is near saturation, and in turn means that any weights input to a saturated input will learn slowly.

A weight will learn slowly if either the input neuron is low activation, or if the output neuron has saturated.

## The Algorithm

* **Input x**: Set the corresponding activation $a^1$ for the input layer.

* **Feedforward**: For each layer compute $z^{l} = w^l a^{l-1}+b^l$ and $a^{l} = \sigma(z^{l})$.

* **Output Error, $\delta^L$**: Compute the vector $\delta^{L}  = \nabla_a C \odot \sigma'(z^L)$.

* **Backpropagate the error**: For each l = L-1, L-2,...,2 compute $\delta^{l} = ((w^{l+1})^T \delta^{l+1}) \odot
  \sigma'(z^{l})$.
  
* **Output**: The gradient of the cost function is given by $\frac{\partial C}{\partial w^l_{jk}} = a^{l-1}_k \delta^l_j$ and $\frac{\partial C}{\partial b^l_j} = \delta^l_j$.

We compute the error vectors backward, starting from the final layer. It is a consequence of the fact that the cost is a function of outputs from the network.